## Gradient Descent Code-Along

Let's walk through how gradient descent works using code.

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
temp = np.random.uniform(-10, 80, 100) # temperature in degrees Fahrenheit

In [10]:
print(np.mean(temp))
print(np.std(temp, ddof = 1))

32.31626690403884
26.77404699137873


In [11]:
# Beers sold at a football stadium(AMERICAN!lol)

In [5]:
beers_sold = 200000 + 1000 * temp + np.random.normal(0, 20000)

In [7]:
beers_sold

array([225449.55206103, 277305.22894166, 257620.39612779, 245620.2049425 ,
       205782.61900458, 205780.44819502, 196968.4664599 , 269696.79448451,
       245841.29242165, 255467.47336641, 193593.54585139, 279032.82805934,
       266660.7790368 , 210851.46132581, 208105.1884134 , 208247.34725157,
       219122.74323112, 238969.02021166, 230615.99304255, 217951.56398259,
       246807.70188978, 204295.38882345, 218033.95973293, 224713.5072612 ,
       232787.2399443 , 262406.77789013, 209711.58175902, 238022.04082199,
       245058.25256235, 195921.47850956, 246419.97803589, 207088.11249662,
       197595.58473344, 277140.63971756, 278647.82434147, 264496.70269524,
       219156.18059037, 200531.43162534, 253321.91375086, 231354.66580133,
       202724.38250079, 236306.86327478, 194835.90826513, 273579.77755185,
       215031.13970876, 251367.94695662, 219794.93821281, 238547.06327077,
       240944.86650566, 208377.84236206, 279003.55786357, 261502.89546726,
       276295.84610554, 2

$$ \text{beers_sold}_i = 200000 + 1000 * \text{temp}_i + \varepsilon_i $$

In [6]:
df = pd.DataFrame({'temp': temp,
                   'beers_sold': beers_sold})

In [7]:
df.head()

,temp,beers_sold
0,23.708611,217728.463687
1,75.564288,269584.140568
2,55.879455,249899.307754
3,43.879264,237899.116568
4,4.041678,198061.530631


#### Our goal is to fit a model here.
- You and I know that our $y$-intercept $\beta_0$ is 200,000.
- You and I know that our slope $\beta_1$ is 1,000.
- However, our computer does not know that. Our computer has to estimate $\hat{\beta}_0$ and $\hat{\beta}_1$ from the data.
    - We might say that our **machine** has to... **learn**.

#### Our workflow:
1. Instantiate model.
2. Select a learning rate $\alpha$.
3. Select a starting point $\hat{\beta}_{1,0}$.
4. Calculate the gradient of the loss function.
5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.
6. Check value of $\left|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}\right|$.
7. Repeat steps 4 through 6 until "stopping condition" is met.

#### Step 1. Instantiate model.

Our model takes on the form:
$$ Y = \beta_0 + \beta_1 X + \varepsilon$$

#### Step 2. Select a learning rate $\alpha$.

$$\alpha = 0.1$$

In [8]:
alpha = 0.001

#### Step 3. Select a starting point.
The zero-th iteration of $\hat{\beta}_1$ is going to start at, say, 20.
$$\hat{\beta}_{1,0} = 20$$

Two points:
- You and I know that the true value of $\beta_1$ is 1000. We need the computer to figure that part out!

#### Step 4. Calculate the gradient of the loss function with respect to parameter $\beta_1$.

The loss function, $L$, is our mean square error.

$$L = \sum_{i = 1} ^ n (y_i - \hat{y}_i)^2 $$

$$\Rightarrow L = \sum_{i = 1} ^ n (y_i - (\hat{\beta}_0 + \hat{\beta}_1x_i))^2 $$

The gradient of this loss function with respect to $\beta_1$ is:

$$\frac{\partial L}{\partial \beta_1} = 2 \sum_{i=1}^n -x_i(y_i - (\hat{\beta}_1x_i + \hat{\beta}_0)) $$

The gradient of this loss function with respect to $\beta_0$ is:
 
$$\frac{\partial L}{\partial \beta_0} =  2 \sum_{i=1}^n -(y_i - (\hat{\beta}_1x_i + \hat{\beta}_0)) $$

In [40]:
temp.shape

(100,)

In [42]:
beers_sold.shape

(100,)

In [9]:
def gradient_descent(alpha, x, y, epsilon = 0.001, max_iter = 10000):
    # setting converged = False 
    converged = False
    # starting at 0 iterations
    iter = 0
    # getting the size of the data
    n = x.shape[0]
    
    #### Step 3. Select a starting point
    beta_0 = 0
    beta_1 = 0
    
    # defining the error function, J
    J = np.sqrt(np.sum([(y[i] - (beta_0 + beta_1 * x[i])) ** 2 for i in range(n)]) )
    
    # Iterate Loop
    while not converged:
        #### Step 4. Calculate the gradient w.r.t the loss function
        grad0 = -2/n * np.sum([(y[i] - (beta_1 * x[i] + beta_0)) for i in range(n)])
        grad1 = -2/n * np.sum([x[i] * (y[i] - (beta_1 * x[i] + beta_0)) for i in range(n)])
        
        #### Step 5. update the weights
       
        beta_0 = beta_0 - alpha * grad0
        beta_1 = beta_1 - alpha * grad1
        
        #### Step 6. Check the stopping condition, in this case, we are checking that the 
        #### error in our model has been reduced to a low level, epsilon(0.001)
        e = np.sqrt(np.sum([(y[i] - (beta_0 + beta_1 * x[i])) ** 2 for i in range(n)]))
        
        if abs(J-e) <= epsilon:
            print ('Converged, iterations: ', iter, '!!!')
            converged = True
        print(abs(J-e))
        J = e   # update error 
        iter += 1  # update iter
    
        if iter == max_iter:
            print ('Max iterations exceeded!')
            converged = True
       
    return beta_0,beta_1
        

In [16]:
gradient_descent(alpha = 0.00015, x = temp, y = beers_sold, epsilon = 0.001, max_iter =100000)

747627.7640792222
225305.82009165478
56217.219986565644
13076.642486286582
3068.2289950749837
805.2984033427201
296.72210013936274
182.56951053417288
156.94437098829076
151.18148715025745
149.87457493809052
149.56731408671476
149.48431509151123
149.45162829314359
149.43023014348
149.41136617516167
149.39307242631912
149.3749083057046
149.3567749718204
149.3386502501089
149.32052916637622
149.30241060396656
149.284294312587
149.26618023542687
149.2480683606118
149.22995868325233
149.21185120404698
149.1937459220644
149.1756428370718
149.15754194837064
149.13944325619377
149.12134676007554
149.10325245955028
149.08516035531648
149.06707044527866
149.04898273106664
149.03089721151628
149.01281388639472
148.9947327550035
148.97665381827392
148.95857707504183
148.94050252484158
148.92243016837165
148.90436000423506
148.8862920331303
148.8682262543589
148.85016266745515
148.8321012724191
148.81404206878506
148.7959850570187
148.7779302361887
148.75987760536373
148.74182716547512
148.72377891

133.5797710272018
133.5635625331197
133.54735600575805
133.53115144441836
133.51494885003194
133.498748220969
133.48254955816083
133.4663528604433
133.4501581285149
133.43396536121145
133.41777455923147
133.4015857214108
133.38539884821512
133.3692139389459
133.35303099337034
133.33685001195408
133.32067099330015
133.30449393810704
133.28831884614192
133.27214571647346
133.25597454910167
133.23980534425937
133.22363810124807
133.20747282006778
133.19130950025283
133.1751481415704
133.1589887442533
133.14283130713739
133.126675831154
133.11052231537178
133.0943707593251
133.07822116324678
133.062073526904
133.04592784983106
133.02978413156234
133.0136423727963
132.99750257190317
132.98136472981423
132.96522884606384
132.9490949199535
132.93296295171604
132.91683294065297
132.90070488699712
132.88457879005
132.86845465027727
132.8523324667476
132.83621223946102
132.82009396818466
132.80397765268572
132.78786329296418
132.77175088832155
132.75564043875784
132.7395319440402
132.72342540370

123.50835464440752
123.49336821015459
123.47838359430898
123.46340079675429
123.44841981725767
123.4334406554699
123.41846331115812
123.40348778420594
123.38851407449692
123.37354218168184
123.35857210552786
123.34360384580214
123.32863740215544
123.31367277482059
123.29870996321552
123.28374896710739
123.26878978637978
123.25383242079988
123.23887687013485
123.22392313415185
123.20897121273447
123.19402110541705
123.17907281219959
123.1641263330821
123.14918166713323
123.13423881470226
123.11929777543992
123.1043585492298
123.08942113548983
123.07448553456925
123.05955174565315
123.04461976897437
123.02968960395083
123.01476125069894
122.99983470886946
122.98490997811314
122.96998705831356
122.9550659492379
122.94014665065333
122.92522916267626
122.91031348449178
122.8953996160999
122.88048755761702
122.86557730834465
122.85066886828281
122.83576223731507
122.82085741497576
122.80595440149773
122.79105319594964
122.77615379856434
122.7612562092254
122.74636042758357
122.7314664531732


112.15009377838578
112.13648554892279
112.12287897116039
112.10927404405084
112.09567076782696
112.08206914213952
112.06846916710492
112.05487084202468
112.04127416701522
112.02767914184369
112.01408576639369
112.00049404019956
111.9869039633777
111.97331553546246
111.95972875633743
111.94614362588618
111.93256014364306
111.91897830995731
111.90539812413044
111.89181958616246
111.87824269547127
111.86466745263897
111.85109385685064
111.83752190787345
111.82395160628948
111.81038295058534
111.79681594180875
111.78325057902839
111.76968686224427
111.75612479145639
111.74256436608266
111.72900558635592
111.7154484516941
111.70189296174794
111.68833911733236
111.67478691681754
111.66123636101838
111.64768744958565
111.63414018193725
111.62059455818962
111.6070505779935
111.59350824146532
111.57996754802298
111.56642849720083
111.5528910899302
111.53935532469768
111.52582120220177
111.51228872162756
111.4987578834407
111.4852286868263
111.47170113201719
111.45817521831486
111.44465094641782

102.96714433818124
102.95465036272071
102.94215790345334
102.92966696014628
102.91717753198463
102.90468961978331
102.89220322249457
102.87971834058408
102.86723497346975
102.85475312091876
102.84227278328035
102.8297939596232
102.81731665018015
102.80484085495118
102.79236657358706
102.77989380538929
102.76742255117279
102.7549528098898
102.74248458165675
102.73001786647364
102.71755266387481
102.70508897374384
102.69262679619715
102.68016613065265
102.66770697711036
102.6552493351046
102.64279320510104
102.63033858628478
102.61788547877222
102.60543388233054
102.5929837966105
102.5805352217285
102.56808815756813
102.55564260331448
102.54319855920039
102.53075602534227
102.51831500104163
102.50587548641488
102.49343748111278
102.48100098513532
102.46856599813327
102.45613252022304
102.44370055035688
102.43127008969896
102.41884113685228
102.40641369251534
102.39398775575683
102.38156332692597
102.36914040567353
102.35671899165027
102.344299085089
102.33188068540767
102.31946379272267


...

0.0014691603023351263
0.0014689820487312488
0.0014688039300665423
0.0014686256958995614
0.0014684473868999959
0.0014682693190923857
0.001468090971059155
0.0014679129568584415
0.0014677348572735838
0.0014675567645401344
0.0014673786578089931
0.0014672006214500044
0.0014670226502033756
0.001466844567858061
0.001466666689395879
0.0014664885791617621
0.0014663107039183387
0.0014661327731158025
0.0014659548758668706
0.0014657769507717688
0.0014655990749279368
0.001465421351337426
0.0014652434478925613
0.001465065724872261
0.0014648879830758688
0.0014647100991158624
0.0014645323970654545
0.0014643546408876773
0.0014641771307903895
0.0014639994034109094
0.0014638216755287203
0.0014636441520341492
0.0014634666131101426
0.0014632888962751167
0.0014631113782108685
0.0014629338061382668
0.0014627562897171487
0.00146257894948576
0.0014624014115689477
0.001462223867557455
0.0014620465293742058
0.0014618691777190662
0.0014616918097942744
0.0014615144754372977
0.0014613371333656033
0.0014611597587759

0.0013014325651283798
0.0013012746765035388
0.0013011167121170786
0.0013009588355021862
0.0013008009217863048
0.0013006431725628431
0.001300485305193888
0.0013003275466623165
0.0013001698403876105
0.0013000118927948279
0.0012998541843245448
0.0012996964897649121
0.0012995387426872895
0.001299381190689175
0.0012992234816522341
0.0012990657380651527
0.0012989082044576605
0.0012987506517241343
0.0012985929517341788
0.001298435393238151
0.0012982778521060112
0.0012981203344750725
0.001297962754197357
0.0012978054304433329
0.0012976478581467887
0.0012974903579010544
0.0012973329754970564
0.0012971755966777465
0.0012970181083655774
0.0012968607272654253
0.0012967033643604964
0.0012965460095397674
0.0012963886655956713
0.0012962314897890792
0.0012960741202405757
0.001295916961655763
0.001295759613748615
0.0012956024351318263
0.0012954452525217874
0.001295288099179004
0.001295130908554043
0.001294973775081587
0.0012948165546102786
0.001294659433858314
0.0012945024089052737
0.001294345250906658

0.0011527136017601691
0.0011525736673885234
0.001152433868545799
0.001152294107253482
0.001152154174627995
0.0011520143889374168
0.0011518746164878024
0.0011517348567107177
0.0011515950770668582
0.0011514553167586428
0.001151315728598945
0.001151175943551408
0.001151036198711708
0.0011508966214943683
0.0011507569827635677
0.0011506172771085232
0.0011504776792978788
0.0011503380829651633
0.0011501984948765198
0.001150058935419196
0.0011499193127590956
0.001149779956804764
0.001149640353460768
0.001149500911175494
0.001149361404744198
0.0011492219773074908
0.0011490825432431961
0.001148942990575108
0.001148803576281665
0.0011486641974780554
0.0011485247856306557
0.0011483855756644346
0.0011482461884071427
0.0011481067650986887
0.0011479675802572586
0.0011478281535666213
0.001147688970252858
0.0011475497235302612
0.001147410531773474
0.0011472712879267988
0.0011471321146920843
0.001146992876659425
0.0011468536725995904
0.0011467144660812778
0.0011465754153334728
0.0011464362041735399
0.00

0.001021113104636484
0.001020989297762398
0.0010208654160113184
0.001020741582856033
0.0010206175483755686
0.0010204938627804694
0.0010203700423829787
0.0010202461641402039
0.0010201223585202257
0.0010199986630201607
0.0010198748151371007
0.001019750979482481
0.001019627316852123
0.0010195036340636676
0.0010193799955651173
0.001019256138267366
0.00101913244304086
0.0010190088388615237
0.0010188851355508177
0.0010187616699912638
0.0010186380095245084
0.0010185143332783753
0.0010183908807768205
0.001018267185562749
0.0010181437306879815
0.0010180200550387042
0.0010178965987588384
0.0010177730883764724
0.001017649622600203
0.001017526129810875
0.0010174026591762697
0.0010172792093996463
0.0010171557081743998
0.0010170324158096378
0.001016908956030349
0.0010167854623315264
0.001016662173917382
0.0010165388494272065
0.001016415406171589
0.0010162921146505965
0.001016168805566764
0.0010160455272085755
0.0010159222204819685
0.0010157989430030767
0.0010156756349619656
0.0010155525235333585
0.0

(194018.5576382134, 1000.0238711979355)

#### Step 5. Calculate $\hat{\beta}_{1,i+1} = \hat{\beta}_{1,i} - \alpha * \frac{\partial L}{\partial \beta_1}$.

#### Step 6. Check value of $\left|\hat{\beta}_{1,i+1} - \hat{\beta}_{1,i}\right|$.

#### Step 7: Save final value of $\hat{\beta}_1$.